# The Sound of a Transiting Exoplanet

:::{warning}
The audio in this notebook may be loud. Please consider the volume on your machine before playing any audio
:::

## Data

The data used in this example is taken from the [Astropy Time Series Usage Example](https://docs.astropy.org/en/stable/timeseries/).

This notebook will not dive into the details about the data, but we will be sonifying the Simple Aperture Photomery Flux (SAP Flux) to listen to the periodic nature of an exoplanet passing in front of its host-star.

The following animation taken from the [Planetary Society](https://www.planetary.org/space-images/light-curve-transit-method) showcase the dip in brightness that we will be listening for.

![animation](https://planetary.s3.amazonaws.com/web/assets/pictures/20191213_cheops-transit-method.gif)

In [ ]:
from astropy.utils.data import get_pkg_data_filename
from astropy.timeseries import TimeSeries

import numpy as np
from astropy import units as u
from astropy.timeseries import BoxLeastSquares
from astropy.stats import sigma_clipped_stats
from astropy.timeseries import aggregate_downsample
import matplotlib.pyplot as plt
import pandas as pd

filename = get_pkg_data_filename('timeseries/kplr010666592-2009131110544_slc.fits')  
ts = TimeSeries.read(filename, format='kepler.fits') 

periodogram = BoxLeastSquares.from_timeseries(ts, 'sap_flux')  

results = periodogram.autopower(0.2 * u.day)  
best = np.argmax(results.power)  
period = results.period[best]   
transit_time = results.transit_time[best]  

mean, median, stddev = sigma_clipped_stats(ts['sap_flux'])  
ts['sap_flux_norm'] = ts['sap_flux'] / median  
ts_binned = aggregate_downsample(ts, time_bin_size=0.02 * u.day)  

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(ts.time.jd, ts['sap_flux'], 'k.', markersize=1)
plt.xlabel('Time (days)')
plt.ylabel('SAP Flux (e-/s)')

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(ts_binned.time_bin_start.jd, ts_binned['sap_flux_norm'], 'k.', markersize=2)
plt.xlabel('Time (days)')
plt.ylabel('SAP Flux (e-/s)')
plt.plot(ts_binned.time_bin_start.jd, ts_binned['sap_flux_norm'], 'r-', drawstyle='steps-post')
plt.xlabel('Time (days)')
plt.ylabel('Normalized flux')

## Sonification

Since our data is already a one-dimensional time series, there isn't any further data processing needed. 

We will, however, store the values in a Pandas Series to take advantage of the Pandas Sonify Accessor.

In [ ]:
import pandas as pd
import scisonify.pandas

sap_flux_norm_binned = pd.Series(ts_binned['sap_flux_norm'].value)
time = ts_binned.time_bin_start.jd

We can now specify our SoundMap, which will be used to turn each data point into a musical note.

In [ ]:
from scisonify.core.soundmaps import DiscreteNoteBins

For this example, we will use 2-octave C major scale. 

In [ ]:
smap = DiscreteNoteBins.from_key("C:maj", octave_range=(3, 4))

Now let's turn our data in music!

We will be listening for dips in frequencies, which signify the exoplanet passing in front of it's source star.

Listening to the whole waveform, we can hear the periodic nature of the brightness dips as the planet revolves around it's star.

In [ ]:
sap_flux_norm_binned.sonify(smap=smap, note_length = 0.10)

We can plot the data points as musical notes to visually see the musical relationship.

In [ ]:
sap_flux_norm_binned.sonify.plot(x=time, ms=1, figsize=(20, 10), title="Normalized & Binned SAP Flux as Musical Notes", xlabel="Julian Date")